In [3]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import numpy as np
import torch
from torch import Tensor
import sigkernel
import os
import sys
import tslearn
import tslearn.metrics
import ksig

from kernels.abstract_base import TimeSeriesKernel, StaticKernel
from kernels.static_kernels import LinearKernel, RBFKernel, PolyKernel
from kernels.integral import StaticIntegralKernel
from kernels.sig_pde import SigPDEKernel
from kernels.sig_trunc import TruncSigKernel
from kernels.gak import GlobalAlignmentKernel, sigma_gak

from features.random_fourier import RBF_RandomFourierFeatures

In [ ]:
#### now implement TRP_RFSF features

class TRP_Gaussian():
    def __init__(
            self, 
            n_features: int, 
            sigma: float,
            d :float #state-space dimension
        ):
        self.n_features = n_features
        self.sigma = sigma

        self.rff_weights = 

In [ ]:
############################################################
#### Test RBF_RandomFourierFeatures vs exact RBF kernel ####
############################################################
def rff_vs_exact_RBFKernel():
    N=3
    N2= 2
    d = 10
    sigma=1
    dtype = torch.float64
    # torch.manual_seed(1)
    X = torch.randn(N, d, dtype=dtype).to("cuda") /np.sqrt(d)
    Y = torch.randn(N2, d, dtype=dtype).to("cuda") / np.sqrt(d)

    # Exact RBF kernel
    k = RBFKernel(sigma=sigma)
    K = k(X, Y)

    # Approximate RBF kernel using RBF_RandomFourierFeatures
    N_MC = 10000
    res = []
    for i in range(N_MC):
        RFF = RBF_RandomFourierFeatures(n_features=1000,
                                        sigma=sigma,
                                        method="cos(x)sin(x)",
                                        # method = "cos(x + b)",
                                        )
        feat_X = RFF(X)
        feat_Y = RFF(Y)
        K_rff = feat_X @ feat_Y.T
        res.append(K_rff)
    K_rff = torch.mean(torch.stack(res), dim=0)

    print("K\n",K)
    print("K_rff\n",K_rff)
    print("diff\n", K-K_rff)
    print("diffmean\n", torch.mean(abs(K-K_rff)))
    # the RFF approach cant reproduce results smaller than 1e-5 for some reason
    
#rff_vs_exact_RBFKernel()

In [ ]:
#####################################
######### FROM KSIG LIBRARY #########
#####################################
import numpy as np
import ksig

def ksig_readme():
    # Number of signature levels to use.
    n_levels = 5 

    # Use 100 components in RFF and projection.
    n_components = 100

    # Instantiate RFF feature map.
    static_feat = ksig.static.features.RandomFourierFeatures(n_components=n_components)
    # Instantiate tensor random projections.
    proj = ksig.projections.TensorizedRandomProjection(n_components=n_components)

    # The RFSF-TRP feature map and kernel. Additionally to working as a callable for
    # computing a kernel, it implements a fit and a transform method.
    rfsf_trp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels, static_features=static_feat, projection=proj)

    # Generate 1000 sequences of length 200 with 100 features.
    n_seq, l_seq, n_feat = 1000, 200, 100
    X = np.random.randn(n_seq, l_seq, n_feat)

    # Fit the kernel to the data.
    rfsf_trp_kernel.fit(X)

    # Compute the kernel matrix as before.
    K_XX = rfsf_trp_kernel(X)  # K_XX has shape (1000, 1000).

    # GEnerate another array of 800 sequences of length 250 and 100 features.
    n_seq2, l_seq2 = 800, 250
    Y = np.random.randn(n_seq2, l_seq2, n_feat)

    # Compute the kernel matrix between X and Y.
    # The kernel does not have to be fitted a second time.
    K_XY = rfsf_trp_kernel(X, Y)  # K_XY has shape (1000, 800)

    # Alternatively, we may compute features separately for X and Y. Under the hood,
    # this is what the call method does, i.e. compute features and take their inner product.
    P_X = rfsf_trp_kernel.transform(X)  # P_X has shape (1000, 501)
    P_Y = rfsf_trp_kernel.transform(Y)  # P_Y shape shape (800, 501)

    # Check that the results match.
    print(np.linalg.norm(K_XX - P_X @ P_X.T))
    print(np.linalg.norm(K_XY - P_X @ P_Y.T))
#ksig_readme()

In [ ]:
#########################
## KSIG test TRP vs DP ##
#########################
import ksig
import numpy as np

def trp_vs_dp():
    n_seq, l_seq, n_feat = 10, 150, 100
    X = np.random.randn(n_seq, l_seq, n_feat)
    n_levels = 10  # Number of signature levels to use.
    n_components = 100 # Use dimension of RFF map

    # Instantiate RFF feature map.
    static_feat = ksig.static.features.RandomFourierFeatures(n_components=n_components)
    trp_proj = ksig.projections.TensorizedRandomProjection(n_components=n_components)
    dp_proj = ksig.projections.DiagonalProjection()

    # The RFSF-TRP feature map and kernel. Additionally to working as a callable for
    # computing a kernel, it implements a fit and a transform method.
    trp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels, 
        static_features=static_feat, 
        projection=trp_proj
        )
    dp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels,
        static_features=static_feat,
        projection=dp_proj
        )
    trp_kernel.fit(X)
    dp_kernel.fit(X)


    # TIME IT
    import timeit
    def function_dp():
        dp_kernel(X, X)
    def function_trp():
        trp_kernel(X, X)
    execution_time_dp = timeit.timeit(function_dp, number=10)
    print("Execution time of dp\t:", execution_time_dp)
    execution_time_trp = timeit.timeit(function_trp, number=10)
    print("Execution time of trp\t:", execution_time_trp)
#trp_vs_dp()